# Iterate over CAD meshes and compare them side by side

In [14]:
import meshzoo
import numpy as np
import igl
import meshplot as mp
import scipy.io
import os
import math
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import Image, display, Markdown
import ipywidgets as widgets
#from meshplot_utils import add_transparent_mesh, color4plot, DrawMeshes, DrawMeshTable, Out, reflectX, cmap_to_numpy
import  meshplot_utils  as mu
#from utils.debug_utils import picture_table_with_title
import utils.debug_utils  as du
import glob
import fnmatch
import  utils.geometry as gu
from auxilary.test_postprocessing import get_obj_files, get_obj_class_by_index, read_cad_mesh, clean_CAD_format
clean_cad_formats =  False

In [15]:
#configurations 
shading_dic = dict(wireframe=True, line_color= [0.2,0.2,0.2] )
resolutions= [2,3,4]

npy_file     = '../auxilary/icoshpere_meshes.npy'
npy_sub_file = '../auxilary/icoshpere_mesh_subdivision_1.npy'


# #========================== small scale tests 10% ==================================
# data_path_list = ['../save/car_10_MRCNN_small/car_small-qualitative',\
#                   '../save/plane_car_1_PointRend_small/aeroplane_small-car_small-qualitative',\
#                   '../save/plane_car_2_PointRend_small/aeroplane_small-car_small-qualitative', \
#                   '../save/bicycle_bus_car_bike_1_PointRend_small/bicycle_small-bus_small-car_small-motorbike_small-qualitative',\
#                   '../save/bicycle_bus_car_bike_4_PointRend_small/bicycle_small-bus_small-car_small-motorbike_small-qualitative' \
#                 ] 


#=============================full test datas ======================================
data_path_list = ['../save/car_10_MRCNN__/car-qualitative',  \
                 '../save/plane_car_1_PointRend__/aeroplane-car-qualitative', \
                 '../save/plane_car_2_PointRend__/aeroplane-car-qualitative', \
                 '../save/bicycle_bus_car_bike_1_PointRend__/bicycle-bus-car-motorbike-qualitative', \
                 '../save/bicycle_bus_car_bike_4_PointRend__/bicycle-bus-car-motorbike-qualitative' \
               ]

CAD_path = "/home/ubuntu/research/datasets/Pascal3D+_release1.1/CAD"
CAD_path_prep = "/home/ubuntu/research/datasets/Pascal3D+_release1.1/CAD_vox_tri"
CAD_classes = ['aeroplane', 'bicycle', 'boat',  'bottle',  'bus',  'car',  'chair',  'diningtable',  'motorbike',  'sofa',  'train',  'tvmonitor']
widgets.Checkbox(value=False,description='show initial shape')
  

#visualize_sphere = True 
sphere_data = []
#if visualize_sphere:
sphere_data = np.load(npy_sub_file, allow_pickle = True).item()
#reflect_sphere_y = True #MCMR intializes shapes with eco-sphere reflected along y axis 
sphere_data['reflect_y'] = True

meshplot_to_html = True
show_sphere_bydefault = True
show_image_table = True #show input and rendered  rgb and mask images 

#enable_index_symmetry     = True  #symmetry of sphere deformation 
#enable_Hausdorff_symmetry = True   #symmetry of reconstructed mesh  (enable only one to avoid  plots being messed up)

enable_rescale_for_ICP = True
load_cad_from_mat      = False
draw_meshes_before_aligment = True 
multi_hist_layout = True
draw_step   = 10
sample_step = 10


## compare CAD loaded from mesh files (obj, or off) to CADs loaded from .mat files 
### (should be the same)

In [16]:
CAD_classes = ['aeroplane']
i = 0 
for cad_class in CAD_classes:
    cad_data = du.loadmat(f'{CAD_path}/{cad_class}.mat')
    subclass_num = cad_data[cad_class].shape[0]
    mu.Out(f'loading {cad_class}.mat : {subclass_num} subclasses',2)
    for ci in range(subclass_num):
        mu.Out(f'  subclasses {ci}',3)
        current_cad = cad_data[cad_class][ci]
        V = current_cad.__dict__['vertices']
        F = current_cad.__dict__['faces'].astype(int) -1 #in mat file vertex indices start from 1 not 0 !
        nF = F.shape[0]
        # print(f'decimating 50 % of {nF} faces')
        #dec_data = tuple(igl.decimate(V,F, nF))
        #print(dec_data)
        #DrawMeshes(None, [ [V,F, [1,.8,.8]] ])
        obj_name = f'{CAD_path}/{cad_class}/{cad_class}_{ci+1:02d}.obj'
        print(f' Save off as  {obj_name}')
        igl.write_obj(obj_name,V,F)
        if i % 5 == 0:
            Vobj, Fobj = igl.read_triangle_mesh(obj_name)
            mu.DrawMeshes(None, [ [V,F, [1,.8,.8]], [ Vobj,Fobj, [1,1,0]] ])
        i += 1

## loading aeroplane.mat : 8 subclasses

###   subclasses 0

 Save off as  /home/ubuntu/research/datasets/Pascal3D+_release1.1/CAD/aeroplane/aeroplane_01.obj


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

###   subclasses 1

...

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

###   subclasses 6

 Save off as  /home/ubuntu/research/datasets/Pascal3D+_release1.1/CAD/aeroplane/aeroplane_07.obj


###   subclasses 7

 Save off as  /home/ubuntu/research/datasets/Pascal3D+_release1.1/CAD/aeroplane/aeroplane_08.obj


# Drawing orignal CADs vs preprocess CADs (voxelized) for each class 

In [17]:
#drawing meshes a aligned table
visualization_step = 1
verbose  = False 
meshplot_to_html = True
#reading CAD meshes from original dataset
CAD_classes = ['car']
mesh_list = []
i = -1 
shading = dict(wireframe=True, line_color= [0.1,0.1,0.1], width=1200, height=2000, wire_width=0.01, flat=True, line_width=0.01)
p = None

for cad_class in CAD_classes:
    if clean_cad_formats:
        clean_CAD_format(CAD_path_prep, [cad_class])
        
    cad_data = du.loadmat(f'{CAD_path}/{cad_class}.mat')
    subclass_num = cad_data[cad_class].shape[0]
    mu.Out(f'loading {cad_class}.mat : {subclass_num} subclasses',2)
    orig_col  =  [0.7,0.7,1, 0.5]
    voxel_col =  [1,1,0, 0.5]
    for ci in range(subclass_num):
        if verbose:
            mu.Out(f'  subclasses {ci}',3)
        i += 1
        if i % visualization_step:
            continue 
        current_cad = cad_data[cad_class][ci]
        Vorig= current_cad.__dict__['vertices']
        Forig = current_cad.__dict__['faces'].astype(int) -1 #in mat file vertex indices start from 1 not 0 !
        nForig = Forig.shape[0]
        Vobj, Fobj, obj_name = read_cad_mesh(CAD_path_prep, cad_class, ci, zero_based_index = True)
        if verbose:
            print(f' reading  {obj_name}')
        Vobj, Fobj = igl.read_triangle_mesh(obj_name)
        mesh_list.append( [Vorig, Forig, orig_col])
        mesh_list.append( [Vobj,  Fobj,  voxel_col])


        #mesh_list += [[Vorig,Forig, [0.7,0.7,1]], [ Vobj,Fobj, [1,1,0]] ]
        #print(Y_shift)
        #p=mu.DrawMeshes(p, [ [Vorig - Y_shift ,Forig, [0.7,0.7,1]], [ Vobj - Y_shift,Fobj, [1,1,0]] ],mesh_shading = shading)
        #Y_shift[1] += 1.1*(Vobj[:,1].max() - Vobj[:,1].min())
    
    ncol = 4
    nrow = int(subclass_num/(0.5*ncol))
    mesh_table = mu.MeshList2Table(mesh_list, nrow, ncol)
    p = mu.DrawMeshTable(None,mesh_table, col_shift=np.array([1.1, 1.5, 1.1, 1.1]),\
                                      row_shift = -1.1, mesh_shading = shading)
    
    
    if meshplot_to_html:
        html_name = f'{cad_class}_CAD.html'
        p.save(html_name)

## loading car.mat : 10 subclasses

Renderer(camera=PerspectiveCamera(aspect=0.6, children=(DirectionalLight(color='white', intensity=0.6, positio…

Plot saved to file car_CAD.html.


## Simple test to pick ICP funciton and show  how well it alignes two meshes 

In [29]:
import igl 

rotate_source = False
source_file = '/home/ubuntu/research/datasets/Pascal3D+_release1.1/CAD_vox_tri/car/car_01_clean_solid_voxrem_dec_smooth_symm_cub_tri.obj'
target_file = '/home/ubuntu/research/datasets/reconstructed_car_sub.obj'



Ver_source, F_source = igl.read_triangle_mesh(source_file)
Ver_source -= np.mean(Ver_source,axis=0)

Ver_target, F_target = igl.read_triangle_mesh(target_file)
Ver_target -= np.mean(Ver_target, axis=0)

alpha = 10 #degrees

#rotating source
if rotate_source:
  Rmat = (R.from_quat([0, 0, np.sin((alpha/180)*np.pi), np.cos((alpha/180)*np.pi)])).as_matrix()
  Ver_source = np.matmul(Ver_source,Rmat)

#Pick ICP function for the test:
#Ver_source_algined, Rot, t = gu.align_source2target_PCA(Ver_source,F_source, Ver_target,F_target, 1000, 10)
#Ver_source_algined, Rot, t = gu.align_source2target_pc_icp(Ver_source,F_source, Ver_target,F_target, 10000, 1, False,False)
Ver_source_algined, Rot, t = gu.align_source2target_with_icp(Ver_source,F_source, Ver_target,F_target, 1000, 10, True)
#Ver_source_algined, Rot, t = gu.align_PC_ICP_init_with_PCA(Ver_source,F_source, Ver_target,F_target, 1000, 10)

print(f' #Vert. = {Ver_source.shape[0]}, {Ver_target.shape[0]}' )
Vs_size = Ver_source.max() - Ver_source.min()
print(f'normalized dist Before ICP = {gu.max_mean_mesh_distance(Ver_source, F_source, Ver_target, F_target)/Vs_size}')
print(f'normalized dist After ICP = {gu.max_mean_mesh_distance(Ver_source_algined, F_source, Ver_target, F_target)/Vs_size}')

shading = dict(wireframe=True, line_color= [0.1,0.1,0.1], width=600, height=600, wire_width=0.01, flat=True, line_width=0.01)
mu.Out('Before and after aligment')
mu.DrawMeshes(None, [ [Ver_source , F_source, [0.5,0.5,1,0.6]], [ Ver_target, F_target, [1,0.5,0.5,0.6]], \
                      [Ver_source_algined , F_source, [0.5,0.5,1,0.6]], [ Ver_target, F_target, [1,0.5,0.5,0.6]]
                    ],shifts =[(0,0,0),(2,0,0), (0,0,0), (0,0,0)],mesh_shading = shading)



 #Vert. = 18718, 2562
normalized dist Before ICP = [0.08942191 0.02574809 0.02304767]
normalized dist After ICP = [0.06542064 0.01189983 0.01113067]


### Before and after aligment

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Testing  open3D PointCloud ICP on  different mesh pairs 
## put  all results in a single  table of 3D plots 

In [19]:
source_target = dict(wireframe=True,  wire_width=0.001, wire_color=mu.rgb2hex(0,0,1), width=1000, height=2000)
target_shadings = dict(wireframe=False)
meshplot_to_html = True
verbose = False


import igl 
import open3d as o3d
from scipy.spatial.transform import Rotation as R
pcad = '/home/ubuntu/research/datasets/Pascal3D+_release1.1/'
s_files = [pcad+'CAD_vox_tri/car/car_01_clean_solid_voxrem_dec_smooth_symm_cub_tri.obj', pcad+'CAD/car/01.off', pcad+'CAD_vox_tri/car/car_01_clean_solid_voxrem_dec_smooth_symm_cub_tri.obj',\
           pcad+'CAD_vox_tri/car/car_01_clean_solid_voxrem_dec_smooth_symm_cub_tri.obj', pcad+'CAD_vox_tri/car/car_01_clean_solid_voxrem_dec_smooth_symm_cub_tri.obj', \
           pcad+'CAD_vox_tri/car/car_01_clean_solid_voxrem_dec_smooth_symm_cub_tri.obj' ]
t_files = [pcad+'CAD_vox_tri/car/car_01_clean_solid_voxrem_dec_smooth_symm_cub_tri.obj', pcad+'CAD/car/01.off', pcad+'CAD/car/01.off',\
           '/home/ubuntu/research/datasets/reconstructed_car_sub.obj',  '/home/ubuntu/research/datasets/reconstructed_car_sub.obj', \
           pcad+'CAD_vox_tri/car/car_07_clean_manual_v4_symm_cub_tri.obj' ]

#s_files = s_files[0:3] #first three pairs 
#t_files = t_files[0:3]

rot_degrees    = [10] *len(s_files) #[10, 10, 10, 0, 10, 10]
transparencies = [10] *len(s_files) #[1,  1, 1,  1, 1, 1] # 10-> opacue , 1-> transparent 
shading_dic = [source_target, target_shadings] * 2*len(s_files)
#print(shading_dic)

mesh_list = []
for i in range(len(s_files)):
#for i in range(2):
   
    s_file = s_files[i] 
    t_file = t_files[i] 
    rot_degree = rot_degrees[i]
    #mu.Out(f'---------  ICP for (source,target,rot) =  {(s_file,t_file,rot_degree)}  -------- ', 2)
    Vs, Fs = igl.read_triangle_mesh(s_file)
    Vt, Ft = igl.read_triangle_mesh(t_file)
    if rot_degree != 0:
        angle = rot_degree*(np.pi/180.0)
        Rmat = (R.from_quat([0, 0, np.sin(angle), np.cos(angle)])).as_matrix()
        Vs = np.matmul(Vs, Rmat)

    Vs_aligned, _, _ = gu.align_open3d_PC_ICP(Vs,Fs, Vt,Ft, 1000, 10, True)
    mu.Out(f'   Source-target dist before ICP = {gu.max_mean_mesh_distance(Vs,Fs,Vt,Ft)}')
    mu.Out(f'   Source-target dist after  ICP = {gu.max_mean_mesh_distance(Vs_aligned,Fs,Vt,Ft)}')

    alpha = transparencies[i]
    s_col = [0.5,0.5,1,0.6*alpha]
    t_col = [1,0.5,0.5,0.9*alpha]

    mesh_list.append([Vs,  Fs, s_col       ] )
    mesh_list.append([Vt, Ft, t_col        ] )
    mesh_list.append([Vs_aligned, Fs, s_col] )
    mesh_list.append([np.copy(Vt), Ft, t_col]         )



mesh_table = mu.MeshList2Table(mesh_list, len(s_files), 4)
p = mu.DrawMeshTable(None,mesh_table, col_shift=np.array([0, 2, 0, 0]),\
                                       row_shift = -1.1, mesh_shading = shading_dic)

if meshplot_to_html:
     html_name = f'ICP_open3d_tests.html'
     p.save(html_name)

###    Source-target dist before ICP = (0.13251367103472422, 0.02373620546602952, 0.02373620546602951)

###    Source-target dist after  ICP = (2.907926436840383e-16, 7.072750444725613e-17, 7.063713558872257e-17)

###    Source-target dist before ICP = (0.13588161066915388, 0.0234507663928262, 0.02331512753120111)

###    Source-target dist after  ICP = (0.0003169758782602093, 3.033121082696859e-08, 3.0331210821814437e-08)

###    Source-target dist before ICP = (0.13512651266370515, 0.033425003547996504, 0.028678835301077725)

...

###    Source-target dist after  ICP = (0.06328391227873738, 0.010819633221875236, 0.009941363211041679)

###    Source-target dist before ICP = (0.13010257453464172, 0.03312400868554808, 0.03253025759988294)

###    Source-target dist after  ICP = (0.08676914110850119, 0.02324920338669507, 0.019521937940555928)

Renderer(camera=PerspectiveCamera(aspect=0.5, children=(DirectionalLight(color='white', intensity=0.6, positio…

Plot saved to file ICP_open3d_tests.html.


## Fix mesh with not  well defined vertex normals
* unit vertex normals are used in my PC-ICP implemntation 
* igl computes  NaN or zero normals for some vertices and - I replace these nomals with  radial directions from vertices to mesh center 

In [26]:
shading = dict(wireframe=False, line_color= [0.1,0.1,0.1], width=600, height=600, wire_width=0.01, flat=True, line_width=0.01)
#higlight vertices with NaN normals
Vt,Ft = igl.read_triangle_mesh('/home/ubuntu/research/datasets/Pascal3D+_release1.1/CAD_vox_tri/car/car_04_clean_manual_symm_cub_tri.obj')

Nt = igl.per_vertex_normals(Vt,Ft)
norm_of_rows = np.linalg.norm(Nt, axis=1)
Nt = Nt/ norm_of_rows[:, np.newaxis]#looks like nt=Nt= unit normals 

nan_index   =  np.any(np.isnan(Nt),axis=1)
valid_index =  np.all( np.logical_not(np.isnan(Nt)),axis=1)
if np.any(nan_index):
    Nt[nan_index,:] = Vt[nan_index,:]
    Nt_fixed = Vt[nan_index,] - np.mean(Vt,axis=0)
    Nt_fixed_norm = np.linalg.norm(Nt_fixed, axis=1)
    Nt_fixed = Nt_fixed/Nt_fixed_norm[:, np.newaxis]
    Nt_fixed[np.isclose(Nt_fixed_norm,0),:] = np.array([1,0,0])  #put arbitary unit norm for 0 vertices with nan normals
    Nt[nan_index,:] = Nt_fixed

p = mu.DrawMeshes(None, [ [Vt,Ft, [0.7,0.7,1, 0.5], nan_index, [1,0,0] ] ],mesh_shading = shading)
p.add_lines(Vt[valid_index,:], Vt[valid_index,:]+0.01*Nt[valid_index,:] ) #valid normal are short 
p.add_lines(Vt[nan_index,:], Vt[nan_index,:]+0.05*Nt_fixed) #fix normals are highlited by large length

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Invalid color array given! Supported are numpy arrays. <class 'numpy.ndarray'>


3